In [19]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the cleaned dataset from Phase 1
df = pd.read_csv('C:/Users/Saisa/Downloads/Walmart_Sales/data/processed/processed_sales.csv', parse_dates=['date'])

# Sort by store and date
df = df.sort_values(by=['store', 'date'])


In [20]:
# Date Parts
df['day'] = df['date'].dt.day
df['week'] = df['date'].dt.isocalendar().week
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter
df['year'] = df['date'].dt.year
df['weekday'] = df['date'].dt.weekday
df['is_weekend'] = df['weekday'].isin([5, 6]).astype(int)



In [21]:
# Lag Features
for lag in [1, 2, 4, 13, 26, 52]:
    df[f'lag_{lag}'] = df.groupby('store')['weekly_sales'].shift(lag)


In [22]:
# Rolling averages
for window in [4, 13, 26, 52]:
    df[f'sales_roll_{window}'] = (
        df.groupby('store')['weekly_sales']
        .shift(1)
        .rolling(window=window)
        .mean()
    )

In [23]:
# Sales difference 
df['sales_diff_1'] = df.groupby('store')['weekly_sales'].diff(1)
df['sales_diff_4'] = df.groupby('store')['weekly_sales'].diff(4)

In [24]:
# Cumulative Sales

df['cumulative_sales'] = df.groupby('store')['weekly_sales'].cumsum()


In [25]:
# Store-wise average sales
df['avg_sales_per_store'] = df.groupby('store')['weekly_sales'].transform('mean')


In [26]:
# Promotion Features
df['promo_last_week'] = df.groupby('store')['is_holiday'].shift(1)
df['promo_next_week'] = df.groupby('store')['is_holiday'].shift(-1)


In [28]:
# Label Encoding

le_store = LabelEncoder()
le_dept = LabelEncoder()

if 'store' in df.columns:
    df['store_encoded'] = le_store.fit_transform(df['store'])

if 'dept' in df.columns:
    df['dept_encoded'] = le_dept.fit_transform(df['dept'])


In [29]:
# Missing Values

df.fillna(0, inplace=True)

In [30]:
output_path = 'C:/Users/Saisa/Downloads/Walmart_Sales/data/processed/feature_engineered_sales.csv'
df.to_csv(output_path, index=False)
print("✅ Feature engineering completed and file saved to:")
print(output_path)

✅ Feature engineering completed and file saved to:
C:/Users/Saisa/Downloads/Walmart_Sales/data/processed/feature_engineered_sales.csv
